In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
recipes_data_df = pd.read_csv('out/recipes_data.csv')

In [3]:
recipes_data_df.head()

,title,year
0,terrina de melón con gelée de oporto,1987
1,mousse de trufa negra '87,1987
2,"ensalada de pasta fresca con caviar, tempura d...",1987
3,"raviolis de cigala, patatas y trufa negra",1987
4,tempura de flor de calabacín rellena de mozzar...,1987


In [4]:
recipes_ml_df = pd.read_csv('out/recipes_ml.csv')

In [5]:
recipes_ml_df.head()

,num_ingredients,num_preparations,num_styles,num_techniques,num_techniquesR,num_worlds,i_chocolate troceado,i_pizza,i_jugo de trufa negra,i_fresitas liofilizadas,...,temp_CALIENTE/FRÍA,temp_HELADA/FRÍA,temp_TIBIA/CALIENTE,temp_FRÍA/HELADA,temp_CALIENTE/HELADA,temp_TIBIA,temp_HELADA/FRÍA/AMBIENTE,temp_TIBIA/AMBIENTE,w_DULCE,w_SALADO
0,7,4,1,17,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,15,5,0,22,0,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
2,21,5,0,29,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,30,10,1,56,0,3,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3
4,25,7,1,37,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
X = recipes_ml_df
y = recipes_data_df.year

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)

In [8]:
y_df = pd.DataFrame(
    {'train': y_train.value_counts(), 'test': y_test.value_counts(), 'total': y.value_counts()},
    columns=['train', 'test', 'total'],
)

In [9]:
y_df

,train,test,total
1987,14,1,15
1988,23,3,26
1989,25,3,28
1990,29,3,32
1991,51,6,57
1992,34,4,38
1993,27,3,30
1994,54,6,60
1995,46,5,51
1996,50,6,56


In [10]:
def my_score(f, xs, ys):
    assert(len(xs) == len(ys))
    a = 18
    b = sum(abs(x - y) for x, y in zip(xs, ys)) / len(xs)
    return (f(a) - f(b)) / f(a)

def my_linear_score(xs, ys):
    return my_score(lambda x: x, xs, ys)

# def my_squared_score(xs, ys):
#     return my_score(math.sqrt, xs, ys)

In [11]:
# %%time

# results = {}
# for clf, params in clfs_params:
#     grid_search_cv = GridSearchCV(clf(), params, cv=10, scoring=make_scorer(my_linear_score), error_score=0, n_jobs=-1)
#     X_train_ = X_train_dict.get(clf.__name__, X_train)
#     y_train_ = y_train
#     grid_search_cv.fit(X_train_, y_train_)
#     results[clf.__name__] = grid_search_cv
#     print(clf.__name__, 'done.')

In [17]:
clf = RandomForestClassifier(class_weight=None, n_estimators=100, n_jobs=-1, random_state=0)

In [18]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [19]:
y_pred = clf.predict(X_test)

In [20]:
my_linear_score(y_pred, y_test)

0.94307832422586513

In [28]:
features_df = pd.DataFrame(
    [X.columns, clf.feature_importances_],
    index=['feature', 'importance']
).T

In [31]:
features_df.sort_values('importance', ascending=False)

,feature,importance
3,num_techniques,0.0132332
5,num_worlds,0.0132191
0,num_ingredients,0.0126246
4,num_techniquesR,0.0122363
2736,w_SALADO,0.00983503
1,num_preparations,0.00942976
2684,t_horno,0.00745607
2334,pf_SALSAS,0.00743707
2487,t_guardar,0.00682988
2,num_styles,0.00674511


In [22]:
len(clf.feature_importances_)

2737

In [23]:
X

,num_ingredients,num_preparations,num_styles,num_techniques,num_techniquesR,num_worlds,i_chocolate troceado,i_pizza,i_jugo de trufa negra,i_fresitas liofilizadas,...,temp_CALIENTE/FRÍA,temp_HELADA/FRÍA,temp_TIBIA/CALIENTE,temp_FRÍA/HELADA,temp_CALIENTE/HELADA,temp_TIBIA,temp_HELADA/FRÍA/AMBIENTE,temp_TIBIA/AMBIENTE,w_DULCE,w_SALADO
0,7,4,1,17,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,15,5,0,22,0,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
2,21,5,0,29,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,30,10,1,56,0,3,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3
4,25,7,1,37,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,29,9,2,46,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
6,20,8,0,29,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
7,25,9,1,37,0,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
8,15,5,0,27,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
9,26,10,0,36,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
